In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Administering Your GIS Organizations Using ArcGIS API for Python

## Overview

<img src="./img/gettingstarted.jpg" width=700/>

- The ArcGIS ecosystem is vast
- Organizations can have multiple versions of any product or multiple products to manage
- How do you manage this?

## The Way of the Python

<img src="./img/waysofscience.jpg" width=700/>

- The Python API allows administrators to manage, update and control what happens on your server
- Script from your favorite IDE or Notebook environment
- Cross platform support

## What Can We Manage?

<table  style='font-family:"Courier New", Courier, monospace; font-size:200%' width=50%>
  
  <tr>
    <td>Users</td>
    <td><img src="./img/users.png", width=50/></td>
  </tr>
  <tr>
    <td>Content</td>
    <td><img src="./img/content.png", width=50/></td>
  </tr>
  <tr>
    <td>Infrastructure</td>
    <td><img src="./img/infrastructure.png", width=50/></td>
  </tr>
  <tr>
    <td>Groups</td>
    <td><img src="./img/groups.png" width=50/></td>
  </tr>
 
</table>

## Getting Started

### Understand the `GIS` Object

The `GIS` object is the way users connect to ArcGIS Online and/or Enterprise

- It doesn't matter if you are an administrator of a user, we must start here.

#### Connecting to you `GIS` 

The ArcGIS API for Python support multiple ways of connecting to the `GIS`, which is ArcGIS Online or ArcGIS Enterprise

##### Anonymously

In [ ]:
import pandas as pd
from arcgis.gis import GIS
gis = GIS() #anonymous connection

##### Built-In

- username/password login method
- usersname are case sensitive 

```python
gis = GIS(username='fakeaccount', password='fakepassword')
gis = GIS(url="https://www.mysite.com/portal", username='fakeaccount', password='fakepassword')
```

**Protecting Built-In Credentials**

- using `profiles` will help protect username and passwords.  
- prevents accidental sharing

1. Create a `GIS` object with the extra `profile` parameter

```python
gis = GIS(url="https://www.mysite.com/portal", 
          username='fakeaccount', 
          password='fakepassword', 
          profile='portal_profile')
```

2. Now connect using the `profile`

```python
gis = GIS(profile='portal_profile')
```

**What Happened?**

Instead of keeping your password in plain text, now we leverage the operating system's credential store for the logged in user.  The credentials never get passed on when you use profiles.

##### Other Login Methods

- LDAP
- IWA
- PKI
- OAuth 2.0.
- API Key

## User Management

<img src="./img/usermanagement.jpg"/>


Users fuel your system. As an administrator your job is to ensure they can put up there content and know the site is reliable and safe.  The Python API is a tool to do just that!

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile', verify_cert=False)

### Working with Existing Users

In [ ]:
um = gis.users
um

#### Search for Users

In [ ]:
users = um.search("geo*")
users

#### List User's Groups

In [ ]:
um.user_groups(um.search("*")[:2])

#### Access a User's Items

In [ ]:
users[0].items()

In [ ]:
user = um.search("geosaurus")[0]
user.update(
    thumbnail=r"./img/dino.png"
)
user

### Creating New Users

In [ ]:
# Check the user types available
um.counts('user_type')

In [ ]:
import uuid
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"
new_user = um.create(username=username, password=password, 
                     firstname="Dan", lastname="Human", 
                     email='testsadf@esri.com')
new_user

In [ ]:
username, password

In [ ]:
new_password = f"!{uuid.uuid4().hex[:8]}A"

#### Reset the Password

In [ ]:
new_user.reset(
    password=password,
    new_password=new_password,
    new_security_question=1,
    new_security_answer=uuid.uuid4().hex[:10],
    reset_by_email=False,
)

In [ ]:
GIS(username=username, 
    password=new_password, 
    use_gen_token=True, 
    verify_cert=False, 
    proxy=detect_proxy(True)).users.me

#### Deleting the User

In [ ]:
new_user.delete()

### Working with Roles and User Types

<img src="./img/know_your_role.jpg" width=500/>

#### User types

- User type determines the privileges that can be granted to the member through a default or custom role
- Common Roles:
  + viewer, creator and administrator

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_enterprise_profile', verify_cert=False)
user = gis.users.me

In [ ]:
user.user_types()['id']

In [ ]:
user.update_license_type("GISProfessionalAdvUT")
user.user_types()['id']

In [ ]:
user.update_license_type("creatorUT")

In [ ]:
user.user_types()['id']

#### Working with Roles

- A role defines the set of privileges assigned to a member

**Accessing Role Manager**

In [ ]:
rm = gis.users.roles
rm

**Listing Roles**

In [ ]:
rm.all()

**Check for Existence of a Role**

In [ ]:
rm.exists('DataEditorRole')

In [ ]:
role = rm.create(name="DataEditorRole", 
                 description="Allow to modify service data", 
                 privileges=[
                        "features:user:edit",
                        "features:user:fullEdit",
                        "opendata:user:designateGroup",
                        "portal:admin:viewUsers",
                        "portal:user:createGroup"]
                )
role

**Removing the Role**

In [ ]:
role.delete()

## Managing Content

In [ ]:
from arcgis.gis import GIS
from arcgis.auth.tools._util import detect_proxy
gis = GIS(profile='your_online_profile', verify_cert=False, proxy=detect_proxy(True))

In [ ]:
cm = gis.content
cm

### Working with Content

<img src="./img/content-manager.jpg"/>

**The content manager allows users and administrators to work with, find and manage content**

#### Searching

##### `search` Example

- provides a simple search method
- max items is 10,000
- do not have full control over searches

In [ ]:
cm.search(query="title: battle", item_type="Feature Layer", outside_org=False)

In [ ]:
cm.search(query="title: battle", item_type="Feature Layer", outside_org=True)

##### `advanced_search` Example

- full control searching option
- removed limitations of `search`
- returns items as dictionary, which speeds up searches
- leverage system for simple statistics about content

**How Many Item to Examine?**

In [ ]:
count = cm.advanced_search('title: battle AND  (type:"feature service")', return_count=True)
count

In [ ]:
items = cm.advanced_search('title: battle AND  (type:"feature service")', max_items=count, 
                           sort_field='avgRating', sort_order='desc')['results']
items[10:20]

**Gathering Information from Searches**

- In this demo we will see how much new content was added to the organization in the last 5 days.

In [ ]:
import datetime as _dt

now =_dt.datetime.now(_dt.timezone.utc)
then = now - _dt.timedelta(days=5)


In [ ]:
cm.advanced_search(
    f"created: [{int(then.timestamp()* 1000)} TO {int(now.timestamp()* 1000)}] AND accountid:{gis.properties.id}", 
    return_count=True)

#### Adding and Publishing Content

In [ ]:
import uuid
username = f"UCUser{uuid.uuid4().hex[:3]}"
password = f"!{uuid.uuid4().hex[:6]}A"
user = gis.users.create(username=username,
        password=password,
        firstname=uuid.uuid4().hex[:6],
        lastname=uuid.uuid4().hex[:6],
        email=uuid.uuid4().hex[:6] + "@esri.com")
user

In [ ]:
user.items()

##### Publishing a Table

In [ ]:
import io, uuid
import pandas as pd

In [ ]:
buffer = io.StringIO()
df = pd.read_csv("./data/banklist.csv")
df.to_csv(buffer)

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

In [ ]:
item_failed_banks = gis.content.add(
        item_properties={
            "type": "CSV",
            "title": "Failed Banks",
            "fileName": f"failedbanks{uuid.uuid4().hex[:5]}.csv",
        },
        data=buffer,
)

In [ ]:
item_failed_banks

In [ ]:
analyzed = gis.content.analyze(item=item_failed_banks)
publish_parameters = analyzed['publishParameters']
publish_parameters[
        'name'
    ] = f"Failed_Banks_{uuid.uuid4().hex[:2]}"  # this needs to be updated
publish_parameters['locationType'] = None  # this makes it a hosted table
published_item = item_failed_banks.publish(publish_parameters)

In [ ]:
published_item

##### Adding/Publishing Content to a User

In [ ]:
user_item = gis.content.add(
    item_properties= {'title' : 'World Cities', 
                      'tags' : "cities, world coverage, population", 
                      'type' : "Shapefile" },
    data=r".\cities.zip",
    owner=user,
)
user_item

In [ ]:
pitem = user_item.publish({'name' : f'world{uuid.uuid4().hex[:4]}cities'})
pitem

##### Reassigning Content 

In [ ]:
user.items()

In [ ]:
folder = gis.content.create_folder("uc_demo_migration_folder")
folder

In [ ]:
item = user.items()[-1]
item.reassign_to(target_owner=gis.users.me.username,
    target_folder= 'uc_demo_migration_folder')

In [ ]:
gis.users.me.items("uc_demo_migration_folder")

##### Erase the User

In [ ]:
user.delete()

## Metadata 

- Administrators can enable metadata for the organization

In [ ]:
gis = GIS(profile='your_online_profile')
mm = gis.admin.metadata
mm

In [ ]:
mm.enable()

In [ ]:
mm.is_enabled

## Content Migration/ Monitoring

### Cloning Items

**`clone_items` provides the ability to copy content from one site to another**

<img src="./img/clone_items.jpg" />

## Continue onto Reporting and Monitoring 